# Simulation Behaviors with LLM based on the context

In [ ]:
from langchain_community.chat_models import OllamaChat
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOllama

In [ ]:
chat_model = ChatOllama(model='phi3:instruct')

In [ ]:
system_propmt = '''
You are an expert analyst of social networks, given the context of connect propobilities with the similar node in the past, evaluate the possibility of different options.
Note:
- Wrap the final answer (the options and the probabilities) in triple backticks (```json ```) to indicate a json block.
- Follw with a list of the reasons for your choice.
'''
user_prompt = '''
Given a {node_type} Node with profile:
{profile}
And the context of possibilities of similar connections in the past:
{old_context}
What the probabilities of the different options in a {new_context} context?
'''

In [ ]:
class JsonOutputParser(StrOutputParser):
    def parse(self, text: str) -> str:
        choice_pattern = re.compile(r'```(?:json)?(.*?)```', re.DOTALL)
        choice_match = choice_pattern.search(text)
        choice = ""
        try:
            choice = choice_match.group(1)
            return json.loads(choice.strip()), text.strip()
        except Exception as e:
            return {},text.strip()

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system",system_propmt),
        ("human", user_prompt),
    ]
)
choice_response = (
    RunnablePassthrough.assign(
        node_type=lambda _: node_type,
        profile=lambda _: profile,
        old_context=lambda _: old_context,
        new_context=lambda _: new_context
    )
    | prompt
    | chat_model.bind()
    | JsonOutputParser()
)

choice_possibility, response = choice_response.invoke({
    "node_type": node_type,
    "profile": profile,
    "old_context": old_context,
    "new_context": new_context
})


In [ ]:
def get_llm_recommendation(node_type, profile, old_context, new_context):
    print(system_propmt)
    print(user_prompt)
    answer = input()
    return answer

### Scenario 1

Actors ---> Actors 

When a new person come to the community who are more likely to connect with him?

In [1]:
from choice_model import ChoiceModel
import networkx as nx
model = ChoiceModel()

In [6]:
period = 1
new_profile = {
'age': '18-35',
'gender': 'Female',
'marrige': 'Yes',
'withkids': 'No',
'student': 'Yes',
'workstatue': 'Freelancer',
'residentinneighbor': 'No',
'educationlevel': 'Undergraduate'
}
model.set_period(period)

choices,probabilities = model.link_prediction(profile=new_profile, k1=10, k2=5, node_type='Actors', choice_type='Actors')

In [7]:
import random

choice = random.choices(choices, probabilities, k=1)[0]
print(choice)

{'type': 'Actors', 'period': 1, 'properties': {'age': '35-55', 'gender': 'Male', 'marrige': 'No', 'withkids': 'No', 'student': 'No', 'workstatue': 'Self-employed', 'residentinneighbor': 'No', 'educationlevel': 'Undergraduate'}, 'label': 'Actors', 'embedding': [-0.3604232668876648, 0.2521228790283203, -2.7096004486083984, -0.5816302299499512, -0.310039758682251, 1.779092788696289, 0.8897720575332642, -0.777116060256958, 0.485622763633728, 0.11924110352993011, -1.2796028852462769, -0.16014087200164795, 1.0756241083145142, 0.3336901068687439, -0.2892748713493347, -0.44426384568214417, 0.23928207159042358, -1.1550233364105225, 0.029033780097961426, 0.6971544623374939, -1.3755970001220703, -0.5991235971450806, -0.44014284014701843, 0.13605117797851562, 1.8435200452804565, 0.5872727036476135, -0.5001554489135742, 0.0837918072938919, -0.32825303077697754, 0.023753374814987183, 0.20175904035568237, 1.172038197517395, 0.7940845489501953, -0.18052785098552704, -1.1734015941619873, -1.24692416191

### Scenario 2

Event ---> Actors

When a new Event happend who are likely to be attracted to the event

In [8]:
period = 1
new_event = {'eventtopic': 'Environment And Sustainability'}
model.set_period(period)

choices,probabilities = model.link_prediction(profile=new_event, k1=10, k2=5, node_type='Event', choice_type='Actors')

In [9]:
import random
choice = random.choices(choices, probabilities, k=1)[0]
print(choice)

{'type': 'Actors', 'period': 0, 'properties': {'age': '35-55', 'gender': 'Female', 'marrige': 'No', 'withkids': 'No', 'student': 'No', 'workstatue': 'Employed', 'residentinneighbor': 'No', 'educationlevel': 'Undergraduate'}, 'label': 'Actors', 'embedding': [-0.03428301215171814, 0.05296250432729721, -2.7666189670562744, -0.6437054872512817, -0.2802574932575226, 2.0165233612060547, 0.5336205959320068, -0.8881096839904785, 0.23439636826515198, 0.23630355298519135, -1.3074488639831543, -0.09264157712459564, 1.3341147899627686, 0.07877178490161896, -0.24775950610637665, -0.7254951596260071, 0.3221322000026703, -1.037096381187439, -0.07340623438358307, 0.5881116986274719, -1.3229105472564697, -0.332059383392334, -0.44045355916023254, 0.2520020008087158, 1.8380050659179688, 0.9758013486862183, -0.45278406143188477, 0.03616414964199066, -0.3651806116104126, 0.13122375309467316, 0.04398781433701515, 0.8282822966575623, 1.071873426437378, -0.3161308169364929, -0.918492317199707, -1.411401510238

### Scenario 3

Actors ---> Event

What event are more like to create by a person with profile

In [10]:
period = 2
new_profile = {
'age': '18-35',
'gender': 'Female',
'marrige': 'Yes',
'withkids': 'No',
'student': 'Yes',
'workstatue': 'Freelancer',
'residentinneighbor': 'No',
'educationlevel': 'Undergraduate'
}
model.set_period(period)

choices,probabilities = model.link_prediction(profile=new_profile, k1=10, k2=5, node_type='Actors', choice_type='Event')

In [11]:
import random

choice = random.choices(choices, probabilities, k=1)[0]
print(choice)

{'type': 'Event', 'period': 2, 'properties': {'eventtopic': 'Technology And Education'}, 'label': 'Event', 'embedding': [-0.656879723072052, 0.91513991355896, -4.235602855682373, -0.8479329347610474, 1.311091661453247, 0.8462489247322083, 0.8574997186660767, -0.08142569661140442, 1.1854817867279053, 0.011350243352353573, 0.9636965394020081, 0.289999395608902, 2.139066457748413, -0.08441591262817383, -0.471753865480423, -0.5701751708984375, -1.3968905210494995, -0.30984020233154297, -1.947365403175354, 1.551087737083435, 0.2809758186340332, -0.7579289674758911, -1.2845512628555298, 0.15801817178726196, 1.1944299936294556, 0.5440708994865417, -0.4083217978477478, -0.02724732644855976, -0.7712206244468689, 0.013595931231975555, 0.5070693492889404, -0.8520241379737854, -0.7004753351211548, 0.30362454056739807, -0.46335098147392273, -0.7117146253585815, 0.1339193731546402, 0.2620813548564911, -0.598717987537384, -0.2891487777233124, 0.9235605001449585, 0.10370858758687973, -0.08469086140394